# 데이터 분석 순서
## 1. 라이브러리 불러오기
## 2. 데이터셋 불러오기
## 3. 의존변수 클래스 만들어주기
## 4. 변수정리하기
## 5. 훈련 데이터와 테스트 데이터로 나누기
## 6. 변수 표준화
## 7. 모델훈련
## 8. 모델 점수 측정하기 F1 스코어로 해보기
## 9. 모델 저장하기  
  

                                
  ## 1. 라이브러리를 불러온다.

In [23]:
import numpy as np
import pandas as pd
np.random.seed(523)

In [24]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import sklearn.metrics
from sklearn.datasets import load_iris
from sklearn.externals import joblib

  
  ## 2. 데이터셋을 불러온다.

In [26]:
#데이터를 불러온다.
data=load_iris()

#데이터를 data.frame으로 만든다.
data_iris=pd.DataFrame(data.data, columns=data.feature_names)

#살펴본다.
data_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [27]:
#데이터 프레임에 타켓변수를 추가해준다.
data_iris["species"]=pd.Categorical.from_codes(data.target, data.target_names)

#살펴본다.
data_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


  ## 3. 의존변수 펙터로 만들어주기(더미화?)

In [30]:
#의존변수를 숫자로 표현할 준비를 한다.
factor = pd.factorize(data_iris['species'])
data_iris.species=factor[0]
definitions=factor[1]
print(data_iris.species.head())
print(definitions)

0    0
1    0
2    0
3    0
4    0
Name: species, dtype: int64
CategoricalIndex(['setosa', 'versicolor', 'virginica'], categories=['setosa', 'versicolor', 'virginica'], ordered=False, dtype='category')


  ## 4. 변수정리하기

In [31]:
#X와 Y로 데이터를 나눈다.
data_iris_x = data_iris.iloc[:,0:4].values
data_iris_y = data_iris.iloc[:,4].values
print('The independent features set: ')
print(data_iris_x[:5,:])
print('The dependent variable: ')
print(data_iris_x[:5])

The independent features set: 
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]
The dependent variable: 
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


  ## 5. 훈련-테스트 데이터로 나누기


In [34]:
train_x, test_x, train_y, test_y = train_test_split(data_iris_x, data_iris_y, test_size=0.7, random_state=523)

  ## 6. 변수 표준화

In [36]:
#스케일러 생성
scaler=StandardScaler()
train_x=scaler.fit_transform(train_x)
test_x=scaler.transform(test_x)

  ## 7. 모델 훈련

In [37]:
#훈련세트로 rf모델을 만든다.
classifier=RandomForestClassifier(n_estimators=2011, criterion='entropy', random_state=523)
classifier.fit(train_x, train_y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2011, n_jobs=1,
            oob_score=False, random_state=523, verbose=0, warm_start=False)

  ## 8. 성능평가
  
  
  

In [42]:
# 테스트 셋으로 모델 성능 측정
pred_y = classifier.predict(test_x)

#Reverse factorize (converting y_pred from 0s,1s and 2s to Iris-setosa, Iris-versicolor and Iris-virginica
reversefactor = dict(zip(range(3),definitions))
test_y = np.vectorize(reversefactor.get)(test_y)
pred_y = np.vectorize(reversefactor.get)(pred_y)

# Making the Confusion Matrix
print(pd.crosstab(test_y, pred_y, rownames=['Actual Species'], colnames=['Predicted Species']))

Predicted Species  setosa  versicolor  virginica
Actual Species                                  
setosa                 37           0          0
versicolor              0          32          0
virginica               0           5         31


In [65]:
print(sklearn.metrics.accuracy_score(test_y, pred_y))
print(sklearn.metrics.f1_score(pred_y, test_y, average=None))

0.9523809523809523
[1.         0.92753623 0.92537313]


In [67]:
print(list(zip(data_iris.columns[0:4], classifier.feature_importances_)))
joblib.dump(classifier, 'simple_rf_model.pkl') 

[('sepal length (cm)', 0.13153891351865257), ('sepal width (cm)', 0.030855456559600845), ('petal length (cm)', 0.37002430247441587), ('petal width (cm)', 0.4675813274473315)]


['simple_rf_model.pkl']